In [ ]:
import pandas as pd
import lpips
import cv2
import torch
import numpy as np
import os
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr
from DISTS_pytorch import DISTS
import pyiqa

In [ ]:
gt_scores_df = pd.read_csv("koniq10k_scores_and_distributions.csv")
max_mos = gt_scores_df["MOS"].values.max()
min_mos = gt_scores_df["MOS"].values.min()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
loss_fn_alex = lpips.LPIPS(net='alex')
D = DISTS().to(device)
nima = pyiqa.create_metric('nima', device=device)
ahiq = pyiqa.create_metric('ahiq', device=device)

In [ ]:
esrgan_lpips_scores = {}
esrgan_lpips_scores["image_name"] = []
esrgan_lpips_scores["MOS"] = []

Чем меньше значение LPIPS, тем ближе изображения.

Требуется сделать преобразование в MOS, которое есть в KonIQ-10k. Из вывода LPIPS кажется, что значения варьируются от 0 до 1 (также в статье присутствует сигмоида, которая выдаёт значения от 0 до 1). Следовательно, преобразование расчитывал по следующей формуле:

$$ (1 - d) = \frac{d_{\text{MOS}} - min}{max - min}, $$
$$ d_{\text{MOS}} = (1 - d) \cdot (max - min) + min $$

In [ ]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_lpips_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = (img0 / 127.5) - 1
    img1 = (img1 / 127.5) - 1

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = loss_fn_alex(img0, img1)
    esrgan_lpips_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)[0, 0, 0, 0]))
    df = pd.DataFrame(esrgan_lpips_scores)
    df.to_csv("ESRGAN_LPIPS_scores.csv")
    count += 1
    if count == 3746:
        break

In [ ]:
real_esrgan_lpips_scores = {}
real_esrgan_lpips_scores["image_name"] = []
real_esrgan_lpips_scores["scores"] = []

In [ ]:
images = os.listdir("../images")
for image_name in tqdm(images):
    real_esrgan_lpips_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"../images/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = (img0 / 127.5) - 1
    img1 = (img1 / 127.5) - 1

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = loss_fn_alex(img0, img1)
    real_esrgan_lpips_scores["scores"].append(float(((1 - d) * (max_score - min_score) + min_score)[0, 0, 0, 0]))
    df = pd.DataFrame(real_esrgan_lpips_scores)
    df.to_csv("Real-ESRGAN_LPIPS_scores.csv")

In [ ]:
esrgan_dists_scores = {}
esrgan_dists_scores["image_name"] = []
esrgan_dists_scores["MOS"] = []

In [ ]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_dists_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0).to(device)
    img1 = torch.tensor(img1).to(device)

    d = D(img0, img1)
    del img0
    del img1
    esrgan_dists_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_dists_scores)
    df.to_csv("ESRGAN_DISTS_scores.csv")
    count += 1
    if count == 3746:
        break

In [ ]:
real_esrgan_dists_scores = {}
real_esrgan_dists_scores["image_name"] = []
real_esrgan_dists_scores["score"] = []

In [ ]:
images = os.listdir("../images")
for image_name in tqdm(images):
    real_esrgan_dists_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"../images/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0).to(device)
    img1 = torch.tensor(img1).to(device)

    d = D(img0, img1)
    del img0
    del img1
    real_esrgan_dists_scores["score"].append(float(((1 - d) * (max_score - min_score) + min_score)))
    df = pd.DataFrame(real_esrgan_dists_scores)
    df.to_csv("Real-ESRGAN_DISTS_scores.csv")

In [ ]:
esrgan_nima_scores = {}
esrgan_nima_scores["image_name"] = []
esrgan_nima_scores["MOS"] = []

In [ ]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_nima_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = nima(img0, img1)
    esrgan_nima_scores["MOS"].append(float(((d / 10) * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_nima_scores)
    df.to_csv("ESRGAN_NIMA_scores.csv")
    count += 1
    if count == 3746:
        break

In [ ]:
real_esrgan_nima_scores = {}
real_esrgan_nima_scores["image_name"] = []
real_esrgan_nima_scores["score"] = []

In [ ]:
images = os.listdir("../images")
for image_name in tqdm(images):
    real_esrgan_nima_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"../images/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = nima(img0, img1)
    real_esrgan_nima_scores["score"].append(float(d))
    df = pd.DataFrame(real_esrgan_nima_scores)
    df.to_csv("Real-ESRGAN_NIMA_scores.csv")

In [ ]:
esrgan_ahiq_scores = {}
esrgan_ahiq_scores["image_name"] = []
esrgan_ahiq_scores["MOS"] = []

In [ ]:
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_ahiq_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = ahiq(img0, img1)
    esrgan_ahiq_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_ahiq_scores)
    df.to_csv("ESRGAN_AHIQ_scores.csv")

In [ ]:
real_esrgan_ahiq_scores = {}
real_esrgan_ahiq_scores["image_name"] = []
real_esrgan_ahiq_scores["score"] = []

In [ ]:
images = os.listdir("../images")
for image_name in tqdm(images):
    real_esrgan_ahiq_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"../images/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = ahiq(img0, img1)
    real_esrgan_ahiq_scores["score"].append(float(d * (max_score - min_score) + min_score))
    df = pd.DataFrame(real_esrgan_ahiq_scores)
    df.to_csv("Real-ESRGAN_AHIQ_scores.csv")

In [ ]:
# df_ESRGAN_lpips = pd.read_csv("ESRGAN_LPIPS_scores.csv")
df_Real_ESRGAN_lpips = pd.read_csv("Real-ESRGAN_LPIPS_scores.csv")
# df_ESRGAN_dists = pd.read_csv("ESRGAN_DISTS_scores.csv")
df_Real_ESRGAN_dists = pd.read_csv("Real-ESRGAN_DISTS_scores.csv")
# df_ESRGAN_nima = pd.read_csv("ESRGAN_NIMA_scores.csv")
df_Real_ESRGAN_nima = pd.read_csv("Real-ESRGAN_NIMA_scores.csv")
df_Real_ESRGAN_ahiq = pd.read_csv("Real-ESRGAN_AHIQ_scores.csv")

correlations = {}

# pearson_esrgan_lpips = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values)[0], 3)
pearson_real_esrgan_lpips = round(pearsonr(gt_scores_df["score_n"].values, df_Real_ESRGAN_lpips["scores"].values)[0], 3)
# pearson_esrgan_dists = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values)[0], 3)
pearson_real_esrgan_dists = round(pearsonr(gt_scores_df["score_n"].values, df_Real_ESRGAN_dists["score"].values)[0], 3)
# pearson_esrgan_nima = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values)[0], 3)
pearson_real_esrgan_nima = round(pearsonr(gt_scores_df["score_n"].values, df_Real_ESRGAN_nima["score"].values)[0], 3)
pearson_real_esrgan_ahiq = round(pearsonr(gt_scores_df["score_n"].values, df_Real_ESRGAN_ahiq["score"].values)[0], 3)

# spearman_esrgan_lpips = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values)[0], 3)
spearman_real_esrgan_lpips = round(spearmanr(gt_scores_df["score_n"].values, df_Real_ESRGAN_lpips["scores"].values)[0], 3)
# spearman_esrgan_dists = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values)[0], 3)
spearman_real_esrgan_dists = round(spearmanr(gt_scores_df["score_n"].values, df_Real_ESRGAN_dists["score"].values)[0], 3)
# spearman_esrgan_nima = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values)[0], 3)
spearman_real_esrgan_nima = round(spearmanr(gt_scores_df["score_n"].values, df_Real_ESRGAN_nima["score"].values)[0], 3)
spearman_real_esrgan_ahiq = round(spearmanr(gt_scores_df["score_n"].values, df_Real_ESRGAN_ahiq["score"].values)[0], 3)

# cosine_esrgan_lpips = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_lpips["MOS"].values)), 3)
# cosine_real_esrgan_lpips = round(np.dot(gt_scores_df["score_n"].values, df_Real_ESRGAN_lpips["score_n"].values) / (np.linalg.norm(gt_scores_df["score_n"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_lpips["score_n"].values)), 3)
# cosine_esrgan_dists = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_dists["MOS"].values)))
# cosine_real_esrgan_dists = round(np.dot(gt_scores_df["score_n"].values[:3746], df_Real_ESRGAN_dists["score_n"].values) / (np.linalg.norm(gt_scores_df["score_n"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_dists["score_n"].values)), 3)
# cosine_esrgan_nima = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_nima["MOS"].values)))
# cosine_real_esrgan_nima = round(np.dot(gt_scores_df["score_n"].values[:3746], df_Real_ESRGAN_nima["score_n"].values) / (np.linalg.norm(gt_scores_df["score_n"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_nima["score_n"].values)), 3)

correlations = {
    "LIPIPS": [
        # f"{pearson_esrgan_lpips}/{spearman_esrgan_lpips}/{cosine_esrgan_lpips}",
        f"{pearson_real_esrgan_lpips}/{spearman_real_esrgan_lpips}"
    ],
    "DISTS": [
        # f"{pearson_esrgan_dists}/{spearman_esrgan_dists}/{cosine_esrgan_dists}",
        f"{pearson_real_esrgan_dists}/{spearman_real_esrgan_dists}"
    ],
    "NIMA": [
        # f"{pearson_esrgan_nima}/{spearman_esrgan_nima}/{cosine_esrgan_nima}",
        f"{pearson_real_esrgan_nima}/{spearman_real_esrgan_nima}"
    ],
    "AHIQ": [
        f"{pearson_real_esrgan_ahiq}/{spearman_real_esrgan_ahiq}"
    ]
}

correlations_df = pd.DataFrame(correlations)
correlations_df.to_csv("correlations.csv")

In [ ]:
correlations_df

In [ ]:
gt_scores = {}
gt_scores["image_name"] = []

images = os.listdir("../images")
for image_name in images:
    gt_scores["image_name"].append(image_name)

gt_scores

In [ ]:
scores = [
    8,
    5,
    9,
    7,
    8,
    4,
    10,
    6,
    5,
    10,
    10,
    6,
    8,
    4,
    9,
    7,
    4,
    7,
    10,
    6,
    5,
    5,
    6,
    7,
    4,
    8,
    9,
    6,
    8,
    7
]

gt_scores["score_n"] = scores
gt_scores

In [ ]:
gt_scores_df = pd.DataFrame(gt_scores)
gt_scores_df.to_csv("gt_scores.csv")

In [ ]:
max_score = gt_scores_df["score_n"].values.max()
min_score = gt_scores_df["score_n"].values.min()